In [ ]:
from collections import OrderedDict
from random import randint

class CacheLine:
    def __init__(self, size):
        self.valid = False
        self.tag = None
        self.data = [0] * size

class LRUCacheLine(CacheLine):
    def __init__(self, size):
        super().__init__(size)
        self.access_order = 0  # To track the order of access for LRU

class LRUCache:
    def __init__(self, num_lines, line_size):
        self.num_lines = num_lines
        self.line_size = line_size
        self.cache = OrderedDict()  # Ordered dictionary to maintain access order

    def extract_address_parts(self, address):
        offset_bits = 2  # Just an example, you may need to adjust based on your cache line size
        tag_bits = 32 - offset_bits  # Assuming 32-bit addresses

        offset_mask = (1 << offset_bits) - 1
        tag_mask = ((1 << tag_bits) - 1) << offset_bits

        offset = address & offset_mask
        tag = (address & tag_mask) >> offset_bits
        return tag, offset

    def read(self, address):
        tag, _ = self.extract_address_parts(address)

        if tag in self.cache:
            # Update access order for LRU
            self.cache.move_to_end(tag)
            return self.cache[tag].data
        else:
            print(f"Cache miss for address {hex(address)}")
            # Simulate fetching data from main memory
            data = [1, 2, 3, 4]  # Replace this with actual data retrieval
            if len(self.cache) >= self.num_lines:
                # Remove the least recently used entry if the cache is full
                self.cache.popitem(last=False)
            self.cache[tag] = LRUCacheLine(self.line_size)
            self.cache[tag].valid = True
            self.cache[tag].tag = tag
            self.cache[tag].data = data
            return data

class TwoLevelCache:
    def __init__(self, l1_cache, l2_cache):
        self.l1_cache = l1_cache
        self.l2_cache = l2_cache

    def read(self, address):
        # First, check L1 cache
        l1_data = self.l1_cache.read(address)

        # If L1 cache miss, check L2 cache
        if not l1_data:
           l2_data = self.l2_cache.read(address)
           return l2_data

        return l1_data

class AssociativeCache:
    def __init__(self, num_lines, line_size):
        self.num_lines = num_lines
        self.line_size = line_size
        self.cache = [CacheLine(line_size) for _ in range(num_lines)]

    def extract_address_parts(self, address):
        offset_bits = 2  # Just an example, you may need to adjust based on your cache line size
        tag_bits = 32 - offset_bits  # Assuming 32-bit addresses

        offset_mask = (1 << offset_bits) - 1
        tag_mask = ((1 << tag_bits) - 1) << offset_bits

        offset = address & offset_mask
        tag = (address & tag_mask) >> offset_bits

        return tag, offset

    def read(self, address):
        tag, _ = self.extract_address_parts(address)

        for cache_line in self.cache:
            if cache_line.valid and cache_line.tag == tag:
                print(f"L2 Cache hit for address {hex(address)}")
                return cache_line.data

        print(f"L2 Cache miss for address {hex(address)}")
        # Simulate fetching data from main memory
        data = [1, 2, 3, 4]  # Replace this with actual data retrieval
        empty_line = next((line for line in self.cache if not line.valid), None)

        if empty_line:
            empty_line.valid = True
            empty_line.tag = tag
            empty_line.data = data
        else:
            print("No empty L2 cache lines available. Consider implementing a replacement policy.")

        return data


class TwoLevelCacheWithLRU(TwoLevelCache):
    def __init__(self, l1_cache_size, l2_cache_size):
        l1_cache = LRUCache(num_lines=l1_cache_size, line_size=2)
        l2_cache = AssociativeCache(num_lines=l2_cache_size, line_size=2)
        super().__init__(l1_cache, l2_cache)

class TwoLevelCacheEvaluator:
    def __init__(self, two_level_cache):
        self.two_level_cache = two_level_cache
        self.hits = 0
        self.misses = 0
        self.replacements = 0  # New metric for replacements

    def simulate_workload(self, addresses):
        for address in addresses:
            random_address = address + randint(0, 10)
            data = self.two_level_cache.read(address)
            if data:
              self.hits += 1
            else:
                self.misses += 1
                self.replacements += 1  # Count replacements for each miss

    def calculate_hit_rate(self):
        total_accesses = self.hits + self.misses
        return self.hits / total_accesses if total_accesses > 0 else 0.0

    def calculate_miss_rate(self):
        total_accesses = self.hits + self.misses
        return self.misses / total_accesses if total_accesses > 0 else 0.0

    def calculate_replacement_rate(self):
        total_accesses = self.hits + self.misses
        return self.replacements / total_accesses if total_accesses > 0 else 0.0

# the workload size
workload_size = 1000
workload_with_misses_lru = [randint(0, 1000) for _ in range(workload_size)]

# Example usage for the Two-Level Cache with LRU
two_level_cache_lru = TwoLevelCacheWithLRU(l1_cache_size=4, l2_cache_size=8)
cache_evaluator_lru = TwoLevelCacheEvaluator(two_level_cache_lru)

cache_evaluator_lru.simulate_workload(workload_with_misses_lru)

hit_rate_lru = cache_evaluator_lru.calculate_hit_rate()
miss_rate_lru = cache_evaluator_lru.calculate_miss_rate()
replacement_rate_lur=cache_evaluator_lru.calculate_replacement_rate()

print("\nTwo-Level Cache with LRU:")
print(f"Hit Rate: {hit_rate_lru:.2%}")
print(f"Miss Rate: {miss_rate_lru:.2%}")
print(f"Replacement Rate: {replacement_rate_lur:.2%}")

In [ ]:
from collections import OrderedDict, deque


class FIFOCacheLine(CacheLine):
    def __init__(self, size):
        super().__init__(size)

class FIFOCache:
    def __init__(self, num_lines, line_size):
        self.num_lines = num_lines
        self.line_size = line_size
        self.cache = deque(maxlen=num_lines)  # Use deque for FIFO replacement policy
        self.replacements = 0  # Counter for replacements

    def extract_address_parts(self, address):
        offset_bits = 2  # Just an example, you may need to adjust based on your cache line size
        tag_bits = 32 - offset_bits  # Assuming 32-bit addresses

        offset_mask = (1 << offset_bits) - 1
        tag_mask = ((1 << tag_bits) - 1) << offset_bits

        offset = address & offset_mask
        tag = (address & tag_mask) >> offset_bits

        return tag, offset

    def read(self, address):
        tag, _ = self.extract_address_parts(address)

        for cache_line in self.cache:
            if cache_line.valid and cache_line.tag == tag:
                print(f"L1 Cache hit for address {hex(address)}")
                return cache_line.data

        print(f"L1 Cache miss for address {hex(address)}")
        # Simulate fetching data from main memory
        data = [1, 2, 3, 4]  # Replace this with actual data retrieval
        if len(self.cache) >= self.num_lines:
            # Remove the oldest entry if the cache is full (FIFO)
            self.cache.popleft()
            self.replacements += 1  # Increment replacements counter
        new_cache_line = FIFOCacheLine(self.line_size)
        new_cache_line.valid = True
        new_cache_line.tag = tag
        new_cache_line.data = data
        self.cache.append(new_cache_line)
        return data

    def get_replacement_count(self):
        return self.replacements


class TwoLevelCacheWithFIFO(TwoLevelCache):
    def __init__(self, l1_cache_size, l2_cache_size):
        l1_cache = FIFOCache(num_lines=l1_cache_size, line_size=4)
        l2_cache = AssociativeCache(num_lines=l2_cache_size, line_size=4)
        super().__init__(l1_cache, l2_cache)

# Example usage for the Two-Level Cache with FIFO
workload_size = 1000
workload_with_misses_fifo = [randint(0, 1000) for _ in range(workload_size)]

two_level_cache_fifo = TwoLevelCacheWithFIFO(l1_cache_size=4, l2_cache_size=8)
cache_evaluator_fifo = TwoLevelCacheEvaluator(two_level_cache_fifo)

cache_evaluator_fifo.simulate_workload(workload_with_misses_fifo)

hit_rate_fifo = cache_evaluator_fifo.calculate_hit_rate()
miss_rate_fifo = cache_evaluator_fifo.calculate_miss_rate()
replacement_rate_fifo = cache_evaluator_fifo.calculate_replacement_rate()

print("\nTwo-Level Cache with FIFO:")
print(f"Hit Rate: {hit_rate_fifo:.2%}")
print(f"Miss Rate: {miss_rate_fifo:.2%}")
print(f"Replacement Rate: {replacement_rate_fifo:.2%}")
